In [9]:
import pandas as pd
import numpy as np

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
sales = pd.read_csv('sales_train_validation.csv')


In [5]:
# melt dataframe to usable format
index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
sales_grid = pd.melt(sales, 
                  id_vars = index_columns, 
                  var_name = 'd', 
                  value_name = 'sales')
sales_grid.d = sales_grid.d.apply(lambda d: int(d.split('_')[1])).apply(pd.to_numeric,downcast='unsigned')

In [12]:
# reduce memory usage
sales_grid = reduce_mem_usage(sales_grid)
cat_columns = ['dept_id','cat_id','store_id','state_id']
for col in cat_columns:
    sales_grid[col] = sales_grid[col].astype('category')
    
sales_grid.info()

Mem. usage decreased to 1724.39 Mb (0.0% reduction)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 8 columns):
 #   Column    Dtype   
---  ------    -----   
 0   id        object  
 1   item_id   object  
 2   dept_id   category
 3   cat_id    category
 4   store_id  category
 5   state_id  category
 6   d         int16   
 7   sales     int16   
dtypes: category(4), int16(2), object(2)
memory usage: 1.3+ GB


In [13]:
calendar = pd.read_csv('calendar.csv')
calendar.d = calendar.d.apply(lambda x: x.split('_')[1]).astype("int16")
calendar = reduce_mem_usage(calendar)

Mem. usage decreased to  0.11 Mb (44.3% reduction)


In [14]:
prices = pd.read_csv('sell_prices.csv')
prices = reduce_mem_usage(prices)

Mem. usage decreased to 130.48 Mb (37.5% reduction)


In [15]:
# save to pickle
sales_grid.to_pickle('sales.pkl')
calendar.to_pickle('calendar.pkl')
prices.to_pickle('prices.pkl')